In [1]:
'''
Run on virtual environment: Snow_env with the following packages installed
***tested on Python 3.9.12, does not work with python 3.10***

Load all dependencies via cmd and pay special attention to version requirements.

Steps for installing virtual env: https://janakiev.com/blog/jupyter-virtual-envs/

1st geopandas: https://towardsdatascience.com/geopandas-installation-the-easy-way-for-windows-31a666b3610f
Here are binaries: https://www.lfd.uci.edu/~gohlke/pythonlibs/
2nd: rioxarray
3rd: rasterio
tables
sklearn
earthpy
numpy 1.22
pyshp 2.1
h5py
tqdm
matplotlib 3.5
seaborn
tensorflow



joblib
pillow
panda
pyproj
shapely
kiwisolver
fonttools
pillow
mpl_toolkits - pip install --user basemap
**git clone https://github.com/matplotlib/matplotlib
cd matplotlib
pip install -e .**
tables
pytables
netCDF4 **https://unidata.github.io/netcdf4-python/

Data Assimilation of snotel data
https://snowex-hackweek.github.io/website/tutorials/geospatial/SNOTEL_query.html

'''

import os
import National_Snow_Model
import NSM_SCA
import pandas as pd
import warnings
from datetime import date, timedelta
warnings.filterwarnings("ignore")

2024-04-22 14:44:46.298929: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-04-22 14:44:46.298951: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
#set path directory
os.getcwd()
os.chdir('..')  # TODO find a way around this, it causes issues when rerunning the code
cwd = os.getcwd()
cwd

'/home/rjohnson18/SWEML'

## Running the NSM to create retrospective datasets



In [25]:
#Had to add folders and files manually - 4-22-2024

#Grab existing files based on water year
prev_year = '2022'
prev_date = prev_year + '-09-24'

#input the new water year of choice
new_year = '2018'
new_date = new_year + '-09-24'

#threshold
threshold = '20.0'

#write code for CSV files

#for h5 files
Region_list = ['N_Sierras',
                       'S_Sierras_High',
                       'S_Sierras_Low',
                       'Greater_Yellowstone',
                       'N_Co_Rockies',
                       'SW_Mont',
                       'SW_Co_Rockies',
                       'GBasin',
                       'N_Wasatch',
                       'N_Cascade',
                       'S_Wasatch',
                       'SW_Mtns',
                       'E_WA_N_Id_W_Mont',
                       'S_Wyoming',
                       'SE_Co_Rockies',
                       'Sawtooth',
                       'Ca_Coast',
                       'E_Or',
                       'N_Yellowstone',
                       'S_Cascade',
                       'Wa_Coast',
                       'Greater_Glacier',
                       'Or_Coast'
                      ]

#dragged this file in from local/Box
SWE_new = {}
for region in Region_list:
    #The below file will serve as a starting poinw
    SWE_new[region] = pd.read_hdf(cwd+f"/Predictions/predictions{prev_year}-09-24.h5", key = region)
    SWE_new[region].rename(columns = {prev_date:new_date}, inplace = True)
    path = f"{cwd}/Predictions/{threshold}"
    if os.path.exists(path) == False:
        os.makedirs(path) 
    SWE_new[region].to_hdf(f"{path}/predictions{new_year}-09-24.h5", key = region)
print('.h5 file complete')
    
#set the ground measures features DF    
obs_old = pd.read_csv(cwd+f"/Data/Pre_Processed_DA/ground_measures_features_{prev_year}-09-24.csv")
obs_old.rename(columns = {'Unnamed: 0':'station_id', prev_date:new_date}, inplace = True)
obs_old.set_index('station_id', inplace = True)
obs_old[new_date] = 0
obs_old.to_csv(cwd+f"/Data/Pre_Processed_DA/ground_measures_features_{new_year}-09-24.csv")

print('Ground measures features df complete')

#load the ground_measures_features meta w/preds
obs_meta_old = pd.read_csv(cwd+f"/Data/Processed/DA_ground_measures_features_{prev_year}-09-24.csv")
obs_meta_old.rename(columns = {'Unnamed: 0':'station_id'}, inplace = True)
obs_meta_old.set_index('station_id', inplace = True)
obs_meta_old['Date'] = new_date
obs_meta_old.to_csv(cwd+f"/Data/Processed/DA_ground_measures_features_{new_year}-09-24.csv")

print('Ground measures features meta df complete')

#Make a submission DF
old_preds = pd.read_csv(cwd+ f"/Predictions/submission_format_{prev_date}.csv")
old_preds['2022-10-01'] = 0
new_preds_date = new_year+'-10-01'
old_preds.rename(columns = {'2022-10-01':new_preds_date}, inplace = True)
old_preds.set_index('cell_id', inplace = True)
old_preds.to_csv(cwd+ f"/Predictions/{threshold}/submission_format_{new_date}.csv")
print('New simulation start files complete')

.h5 file complete
Ground measures features df complete
Ground measures features meta df complete
New simulation start files complete


In [26]:
#can be altered to create list every n number of days by changing 7 to desired skip length
def daterange(start_date, end_date):
     for n in range(0, int((end_date - start_date).days) + 1, 7):
        yield start_date + timedelta(n)
         
#define start and end date for list of dates
start_dt = date(2018, 10, 1)
end_dt = date(2019, 7, 31)

#create empty list to store dates
datelist = []

#append dates to list
for dt in daterange(start_dt, end_dt):
    #print(dt.strftime("%Y-%m-%d"))
    dt=dt.strftime('%Y-%m-%d')
    datelist.append(dt)
    
datelist

['2018-10-01',
 '2018-10-08',
 '2018-10-15',
 '2018-10-22',
 '2018-10-29',
 '2018-11-05',
 '2018-11-12',
 '2018-11-19',
 '2018-11-26',
 '2018-12-03',
 '2018-12-10',
 '2018-12-17',
 '2018-12-24',
 '2018-12-31',
 '2019-01-07',
 '2019-01-14',
 '2019-01-21',
 '2019-01-28',
 '2019-02-04',
 '2019-02-11',
 '2019-02-18',
 '2019-02-25',
 '2019-03-04',
 '2019-03-11',
 '2019-03-18',
 '2019-03-25',
 '2019-04-01',
 '2019-04-08',
 '2019-04-15',
 '2019-04-22',
 '2019-04-29',
 '2019-05-06',
 '2019-05-13',
 '2019-05-20',
 '2019-05-27',
 '2019-06-03',
 '2019-06-10',
 '2019-06-17',
 '2019-06-24',
 '2019-07-01',
 '2019-07-08',
 '2019-07-15',
 '2019-07-22',
 '2019-07-29']

## Run model

In [30]:
#run the model through all time (data acqusition already completed)
for date in datelist:
    print('Updating SWE predictions for ', date)
    #connect interactive script to Wasatch Snow module
    Snow = NSM_SCA.NSM_SCA(cwd, date, threshold=0.2)
    
    #Go get SNOTEL observations -- currently saving to csv, change to H5,
    #dd if self.data < 11-1-2022 and SWE = -9999, 
    #Snow.Get_Monitoring_Data_Threaded()

    #Get the prediction extent
    bbox = Snow.getPredictionExtent()

    #Initialize/Download the granules
    Snow.initializeGranules(bbox, Snow.SCA_folder)

    #Process observations into Model prediction ready format,
    Snow.Data_Processing()

    #Agument with SCA
    Snow.augmentPredictionDFs()

    #Make predictions
    Snow.SWE_Predict()

    #Make CONUS netCDF file, compressed.
    #Snow.netCDF_compressed(plot=False)

    #Make GeoDataframe and plot, self.Geo_df() makes the geo df
    # Snow.Geo_df()
    # Snow.plot_interactive_SWE_comp(pinlat = 39.1, pinlong = -120, web = True)

Updating SWE predictions for  2018-10-01
Missing 7 granules for 2018-09-28, downloading
Order:  1


HTTPError: 401 Client Error: Unauthorized for url: https://urs.earthdata.nasa.gov/oauth/authorize?app_type=401&client_id=PGVMJ5nUzSnQkI5o23gMxA&response_type=code&redirect_uri=https%3A%2F%2Fn5eil02u.ecs.nsidc.org%2FOPS%2Fredirect&state=aHR0cHM6Ly9uNWVpbDAydS5lY3MubnNpZGMub3JnL2VnaS9yZXF1ZXN0P3Nob3J0X25hbWU9Vk5QMTBBMUYmdmVyc2lvbj0yJnRlbXBvcmFsPTIwMTgtMDktMjhUMDAlM0EwMCUzQTAwWiUyQzIwMTgtMDktMjhUMjMlM0E1OSUzQTU5WiZwYWdlX3NpemU9MjAwMCZib3VuZGluZ19ib3g9LTEyMy4zNDA3ODUzMDk2MTQ4JTJDMzMuMzU4MjUzNzg2MzA0ODElMkMtMTA1LjA3ODAzNTU4MzQ2NDklMkM0OC45NzEwNjU3MDgwNzk2NSZwcm9qZWN0aW9uPUdFT0dSQVBISUMmZm9ybWF0PUdlb1RJRkYmYWdlbnQ9JmVtYWlsPXJqb2huc29uMTglNDB1YS5lZHUmQ292ZXJhZ2U9JTJGVklJUlNfR3JpZF9JTUdfMkQlMkZDR0ZfTkRTSV9Tbm93X0NvdmVyJnJlcXVlc3RfbW9kZT1hc3luYw

In [8]:
Snow.threshold

20.0

In [ ]:
def run(cwd, date):
    print('Updating SWE predictions for ', date)
    #connect interactive script to Wasatch Snow module
    Snow = NSM_SCA.NSM_SCA(cwd, date)

    #Go get SNOTEL observations -- currently saving to csv, change to H5,
    #dd if self.data < 11-1-2022 and SWE = -9999,
    Snow.Get_Monitoring_Data_Threaded()

    #Get the prediction extent
    bbox = Snow.getPredictionExtent()

    #Initialize/Download the granules
    Snow.initializeGranules(bbox, Snow.SCA_folder)

    #Process observations into Model prediction ready format,
    Snow.Data_Processing()

    #Agument with SCA
    Snow.augmentPredictionDFs()

    #Make predictions
    Snow.SWE_Predict()

    #Make CONUS netCDF file, compressed.
    Snow.netCDF_compressed(plot=False)

In [ ]:
for date in datelist[:4]:
    %time run(cwd, date)